In [16]:
import pandas as pd
import numpy as np

In [17]:
# Our human kinases containing CREs
human_kinases = pd.read_csv('../datasets/human_kinases.txt', names= ["uniprot"], skiprows= 1)

In [18]:
# Cosmic drug resistance mutations
cosmic_resistance = pd.read_csv('../raw_data/Cosmic_ResistanceMutations_v98_GRCh38.tsv.gz', compression= 'gzip', sep= '\t')
cosmic_resistance.columns = cosmic_resistance.columns.str.lower()
cosmic_resistance

,sample_name,cosmic_sample_id,gene_symbol,cosmic_gene_id,transcript_accession,census_gene,drug_name,genomic_mutation_id,legacy_mutation_id,mutation_id,...,pubmed_pmid,cosmic_study_id,mutation_zygosity,chromosome,genome_start,genome_stop,strand,hgvsp,hgvsc,hgvsg
0,1000815,COSS1000815,EGFR,COSG73399,ENST00000275493.6,Yes,Gefitinib,COSV51765492,COSM6240,101848535,...,16983123,NaN,NaN,7,55181378.0,55181378.0,+,ENSP00000275493.2:p.Thr790Met,ENST00000275493.6:c.2369C>T,7:g.55181378C>T
1,1000815,COSS1000815,EGFR,COSG73399,ENST00000454757.6,Yes,Gefitinib,COSV51765492,COSM6240,133415912,...,16983123,NaN,NaN,7,55181378.0,55181378.0,+,ENSP00000395243.3:p.Thr745Met,ENST00000454757.6:c.2234C>T,7:g.55181378C>T
2,1000815,COSS1000815,EGFR,COSG73399,ENST00000455089.5,Yes,Gefitinib,COSV51765492,COSM6240,134089990,...,16983123,NaN,NaN,7,55181378.0,55181378.0,+,ENSP00000415559.1:p.Thr745Met,ENST00000455089.5:c.2234C>T,7:g.55181378C>T
3,1000815,COSS1000815,EGFR,COSG73399,ENST00000638463.1,Yes,Gefitinib,COSV51765492,COSM6240,175838063,...,16983123,NaN,NaN,7,55181378.0,55181378.0,+,NaN,ENST00000638463.1:c.*359C>T,7:g.55181378C>T
4,1000816,COSS1000816,EGFR,COSG73399,ENST00000275493.6,Yes,Gefitinib,COSV51765492,COSM6240,101848535,...,16983123,NaN,NaN,7,55181378.0,55181378.0,+,ENSP00000275493.2:p.Thr790Met,ENST00000275493.6:c.2369C>T,7:g.55181378C>T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,WHIM23,COSS2367162,ESR1,COSG61326,ENST00000456483.3,Yes,Endocrine therapy,COSV52783938,COSM1074639,134118141,...,24055055,NaN,NaN,6,152098788.0,152098788.0,+,NaN,ENST00000456483.3:c.*485A>C,6:g.152098788A>C
5090,basal_cfDNA,COSS2811748,KDR,COSG106784,ENST00000263923.5,Yes,Cetuximab,COSV99819063,COSM9179438,102524718,...,29588308,NaN,het,4,55097758.0,55097758.0,-,ENSP00000263923.4:p.Leu840Phe,ENST00000263923.5:c.2518C>T,4:g.55097758G>A
5091,basal_cfDNA,COSS2811748,KDR,COSG106784,ENST00000645273.1,Yes,Cetuximab,COSV99819063,COSM9179438,179308189,...,29588308,NaN,het,4,55097758.0,55097758.0,-,ENSP00000495159.1:p.Leu840Phe,ENST00000645273.1:c.2518C>T,4:g.55097758G>A
5092,metastatis_DNA,COSS2811749,KDR,COSG106784,ENST00000263923.5,Yes,Cetuximab,COSV99819063,COSM9179438,102524718,...,29588308,NaN,het,4,55097758.0,55097758.0,-,ENSP00000263923.4:p.Leu840Phe,ENST00000263923.5:c.2518C>T,4:g.55097758G>A


In [19]:
# Transcripts stable for mapping uniprot accessions
transcripts = cosmic_resistance.transcript_accession.apply(lambda x: x.split(".")[0])
transcripts = transcripts.drop_duplicates().to_list()
# with open("../datasets/resistance_transcripts.txt", "w") as f:
#     for transc in transcripts:
#         f.write(transc)
#         f.write("\n")

In [20]:
# Mapping transcript_stable - uniprot, obtained from biomart
mapping_transcript_uniprot = pd.read_csv('../datasets/mapping_resistance_transcript_uniprot.tsv', sep= '\t', names= ["gene_name", "uniprot", "transcript_stable", "trembl"], skiprows= 1)
mapping_transcript_uniprot.uniprot = mapping_transcript_uniprot.uniprot.fillna(mapping_transcript_uniprot.trembl)
mapping_transcript_uniprot.drop(columns= 'trembl', inplace= True)
mapping_transcript_uniprot = mapping_transcript_uniprot.drop_duplicates()
mapping_transcript_uniprot

,gene_name,uniprot,transcript_stable
0,FLT3,P36888,ENST00000241453
2,RB1,P06400,ENST00000267163
7,NaN,A0A0B4J203,ENST00000507166
8,JAK2,O60674,ENST00000381652
11,PDGFRA,P16234,ENST00000257290
...,...,...,...
1428,KRAS,G3V4K2,ENST00000556131
1438,NF1,P21359,ENST00000356175
1450,NF1,P21359,ENST00000358273
1462,JAK1,P23458,ENST00000342505


In [21]:
mapping_transcript_uniprot = mapping_transcript_uniprot[mapping_transcript_uniprot.uniprot.isin(human_kinases.uniprot)]
mapping_transcript_uniprot

,gene_name,uniprot,transcript_stable
0,FLT3,P36888,ENST00000241453
11,PDGFRA,P16234,ENST00000257290
186,MAP2K1,Q02750,ENST00000307102
199,KDR,P35968,ENST00000263923
239,KIT,P10721,ENST00000288135
305,FGFR4,P22455,ENST00000502906
312,FGFR4,P22455,ENST00000292408
319,FGFR4,P22455,ENST00000393637
326,FGFR1,P11362,ENST00000397091
380,FGFR1,P11362,ENST00000447712


In [22]:
mapping_transcript_uniprot.uniprot.nunique()

12

In [23]:
cosmic_resistance = cosmic_resistance[['gene_symbol', 'transcript_accession', 'census_gene', 'drug_name', 'mutation_id',
       'mutation_aa', 'cosmic_phenotype_id']].drop_duplicates()
cosmic_resistance["transcript_stable"] = cosmic_resistance.transcript_accession.apply(lambda x: x.split(".")[0])
cosmic_resistance

,gene_symbol,transcript_accession,census_gene,drug_name,mutation_id,mutation_aa,cosmic_phenotype_id,transcript_stable
0,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29974826,ENST00000275493
1,EGFR,ENST00000454757.6,Yes,Gefitinib,133415912,p.T745M,COSO29974826,ENST00000454757
2,EGFR,ENST00000455089.5,Yes,Gefitinib,134089990,p.T745M,COSO29974826,ENST00000455089
3,EGFR,ENST00000638463.1,Yes,Gefitinib,175838063,p.?,COSO29974826,ENST00000638463
12,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29914826,ENST00000275493
...,...,...,...,...,...,...,...,...
5087,ESR1,ENST00000440973.5,Yes,Endocrine therapy,130283412,p.Y537S,COSO28395278,ENST00000440973
5088,ESR1,ENST00000443427.5,Yes,Endocrine therapy,131218810,p.Y537S,COSO28395278,ENST00000443427
5089,ESR1,ENST00000456483.3,Yes,Endocrine therapy,134118141,p.?,COSO28395278,ENST00000456483
5090,KDR,ENST00000263923.5,Yes,Cetuximab,102524718,p.L840F,COSO28864826,ENST00000263923


In [24]:
# Find resistance mutations in our kinases
kinases_resistance = cosmic_resistance.merge(mapping_transcript_uniprot)#.uniprot.nunique()
kinases_resistance

,gene_symbol,transcript_accession,census_gene,drug_name,mutation_id,mutation_aa,cosmic_phenotype_id,transcript_stable,gene_name,uniprot
0,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533
1,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29914826,ENST00000275493,EGFR,P00533
2,EGFR,ENST00000275493.6,Yes,Gefitinib,101851276,p.D761Y,COSO29914826,ENST00000275493,EGFR,P00533
3,EGFR,ENST00000275493.6,Yes,Gefitinib,181387443,p.T790M,COSO29914826,ENST00000275493,EGFR,P00533
4,EGFR,ENST00000275493.6,Yes,Erlotinib,181387443,p.T790M,COSO29914826,ENST00000275493,EGFR,P00533
...,...,...,...,...,...,...,...,...,...,...
730,FGFR2,ENST00000457416.6,Yes,Endocrine therapy,135884404,p.M538I,COSO283910667,ENST00000457416,FGFR2,P21802
731,FGFR2,ENST00000457416.6,Yes,Endocrine therapy,135881262,p.N550K,COSO283910667,ENST00000457416,FGFR2,P21802
732,FGFR2,ENST00000457416.6,Yes,AZD4547,135913986,p.V565F,COSO36006237,ENST00000457416,FGFR2,P21802
733,FGFR2,ENST00000457416.6,Yes,INCB054828,135882909,p.N550H,COSO27814826,ENST00000457416,FGFR2,P21802


In [14]:
classification = pd.read_csv('../raw_data/Cosmic_Classification_v98_GRCh38.tsv', sep= '\t')
classification.columns = classification.columns.str.lower()
classification

,cosmic_phenotype_id,primary_site,site_subtype_1,site_subtype_2,site_subtype_3,primary_histology,histology_subtype_1,histology_subtype_2,histology_subtype_3,nci_code,efo
0,COSO27985024,haematopoietic_and_lymphoid_tissue,NS,NS,NS,haematopoietic_neoplasm,acute_myeloid_leukaemia,M5b,NS,C3171,http://www.ebi.ac.uk/efo/EFO_0000222
1,COSO28974826,small_intestine,duodenum,NS,NS,carcinoma,adenocarcinoma,NS,NS,C7889,http://www.ebi.ac.uk/efo/EFO_1000223
2,COSO36286727,thyroid,NS,NS,NS,carcinoma,papillary_carcinoma,follicular_variant,NS,C7381,http://www.ebi.ac.uk/efo/EFO_1000261
3,COSO27606716,haematopoietic_and_lymphoid_tissue,lymph_node,NS,NS,lymphoid_neoplasm,T_cell_large_granular_lymphocytic_leukaemia,NS,NS,C4664,NaN
4,COSO29645585,kidney,NS,NS,NS,carcinoma,papillary_renal_cell_carcinoma,Type_2,NS,C6975,http://www.ebi.ac.uk/efo/EFO_0000640
...,...,...,...,...,...,...,...,...,...,...,...
6942,COSO286920171,large_intestine,colon,sigmoid,NS,other,carcinoma_in_situ,NS,NS,C3638,NaN
6943,COSO541011753,soft_tissue,nerve_sheath,axilla,NS,malignant_peripheral_nerve_sheath_tumour,epithelioid,NS,NS,C6561,NaN
6944,COSO1098819012,soft_tissue,smooth_muscle,parafallopian_tube,NS,leiomyoma,intravascular_leiomyomatosis,NS,NS,C4518,NaN
6945,COSO380710727,central_nervous_system,parietal_lobe,NS,NS,glioma,astroblastoma,NS,NS,C4324,NaN


In [25]:
classification = pd.read_csv('../raw_data/Cosmic_Classification_v98_GRCh38.tsv', sep= '\t')
classification.columns = classification.columns.str.lower()
classification = classification[['cosmic_phenotype_id', 'primary_site', 'primary_histology']].drop_duplicates()
# Add primary site
kinases_resistance = kinases_resistance.merge(classification)
kinases_resistance

,gene_symbol,transcript_accession,census_gene,drug_name,mutation_id,mutation_aa,cosmic_phenotype_id,transcript_stable,gene_name,uniprot,primary_site,primary_histology
0,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma
1,EGFR,ENST00000275493.6,Yes,Erlotinib,181387443,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma
2,EGFR,ENST00000275493.6,Yes,Gefitinib,181387443,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma
3,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29914826,ENST00000275493,EGFR,P00533,lung,carcinoma
4,EGFR,ENST00000275493.6,Yes,Gefitinib,101851276,p.D761Y,COSO29914826,ENST00000275493,EGFR,P00533,lung,carcinoma
...,...,...,...,...,...,...,...,...,...,...,...,...
730,FGFR4,ENST00000393637.5,Yes,Fisogatinib,181395582,p.C512F,COSO29825606,ENST00000393637,FGFR4,P22455,liver,carcinoma
731,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143403817,p.V550L,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma
732,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143404311,p.V550L,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma
733,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143403745,p.V550M,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma


In [26]:
# Replace "haematopoietic_and_lymphoid_tissue" with values in histology_subtype_1 col
kinases_resistance["tissue"] = np.where(kinases_resistance.primary_site == 'haematopoietic_and_lymphoid_tissue', kinases_resistance.primary_histology, kinases_resistance.primary_site)
kinases_resistance

,gene_symbol,transcript_accession,census_gene,drug_name,mutation_id,mutation_aa,cosmic_phenotype_id,transcript_stable,gene_name,uniprot,primary_site,primary_histology,tissue
0,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma,lung
1,EGFR,ENST00000275493.6,Yes,Erlotinib,181387443,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma,lung
2,EGFR,ENST00000275493.6,Yes,Gefitinib,181387443,p.T790M,COSO29974826,ENST00000275493,EGFR,P00533,lung,carcinoma,lung
3,EGFR,ENST00000275493.6,Yes,Gefitinib,101848535,p.T790M,COSO29914826,ENST00000275493,EGFR,P00533,lung,carcinoma,lung
4,EGFR,ENST00000275493.6,Yes,Gefitinib,101851276,p.D761Y,COSO29914826,ENST00000275493,EGFR,P00533,lung,carcinoma,lung
...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,FGFR4,ENST00000393637.5,Yes,Fisogatinib,181395582,p.C512F,COSO29825606,ENST00000393637,FGFR4,P22455,liver,carcinoma,liver
731,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143403817,p.V550L,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma,liver
732,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143404311,p.V550L,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma,liver
733,FGFR4,ENST00000502906.5,Yes,Fisogatinib,143403745,p.V550M,COSO29825606,ENST00000502906,FGFR4,P22455,liver,carcinoma,liver


In [27]:
kinases_resistance[['primary_site', 'tissue']].value_counts()

primary_site                        tissue                 
haematopoietic_and_lymphoid_tissue  haematopoietic_neoplasm    311
soft_tissue                         soft_tissue                127
lung                                lung                        95
biliary_tract                       biliary_tract               51
breast                              breast                      49
haematopoietic_and_lymphoid_tissue  lymphoid_neoplasm           31
skin                                skin                        24
liver                               liver                       18
NS                                  NS                          10
stomach                             stomach                     10
large_intestine                     large_intestine              9
dtype: int64

In [28]:
kinases_resistance['tissue'].value_counts()

haematopoietic_neoplasm    311
soft_tissue                127
lung                        95
biliary_tract               51
breast                      49
lymphoid_neoplasm           31
skin                        24
liver                       18
NS                          10
stomach                     10
large_intestine              9
Name: tissue, dtype: int64

Ideas:  
- map mutations by binding sites (compare ATP, active site, and rest of KD)
- mutational rates

---

In [42]:
# pdb_sequences = pd.read_csv('../datasets/pdb_sequences.tsv', sep= '\t')
# pdb_sequences

# msa_regions_pdbs = pd.read_csv('../datasets/msa_regions_pdbs.tsv', sep = '\t')
# msa_regions_pdbs

# pdbs = msa_regions_pdbs[msa_regions_pdbs.msa == "P00519_60"].pdb

# p00519 = pdb_sequences[pdb_sequences.pdb.isin(pdbs)]
# p00519["length"] = p00519.sequence.apply(lambda x: len(x))
# p00519 = p00519.sort_values("length")
# p00519